# Question 1: AutoEncoders


## Task :

<ul>
<li>DAC: Deep Autoencoder-based Clustering.</li>
</ul>


## Suggested Model

Figure 1 shows an overview of our deep autoencoder-based clustering framework. There
are two main steps: training and clustering testing. In the training step, a deep autoencoder with an encoder and a decoder is trained using the training set. Here a flattened
input vector is fed into the multilayer deep encoder which has a low dimensional learned
representation. This learned representation is further fed into a decoder that tries to recover an output of the same size as the input. The training process of this autoencoder
tries to reconstruct the input as much as possible. In the following clustering step, we
apply the autoencoder to the testing set. The output of the encoder (learned representations) is then fed to a classic K-Means algorithm to do clustering. The learned low
dimensional representation vector contains key information of the given input, and thus
yield better clustering results.

- Source-paper: DAC: Deep Autoencoder-based Clustering, a General Deep Learning Framework of Representation Learning


<img src="Images/Q3-model.png" width = 500>


## Importing libraries


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch for Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# TensorFlow and Keras for Deep Learning
import tensorflow as tf
from tensorflow import keras

# scikit-learn for clustering
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score


## 1.Designing Deep Auto Encoder Cluster Model ( based on given figure )


In [14]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim=784, encoded_dim=10):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 32),
            nn.Tanh(),
            nn.Linear(32, encoded_dim)  # Latent space representation
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoded_dim, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 512),
            nn.Tanh(),
            nn.Linear(512, 512),
            nn.Tanh(),
            nn.Linear(512, input_dim),
            nn.Sigmoid()  # Sigmoid activation for final output
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded


# Example usage
input_dim = 784  # MNIST images are 28x28 pixels
encoded_dim = 10

autoencoder = Autoencoder(input_dim, encoded_dim)


## 2. Import, Normalize, and Train Data


In [21]:
# Define a transform to normalize the data to the range [0, 1]
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0,), (1,))])

# Download and load the training dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True)

# Download and load the test dataset
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=False)


In [35]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.003)

# Training loop
num_epochs = 20
sample_size = 1000  # Number of samples to use for training

for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, _ = batch
        inputs = inputs.view(inputs.size(0), -1)

        # Limit the number of samples for training
        if sample_size is not None and len(inputs) > sample_size:
            inputs = inputs[:sample_size]

        optimizer.zero_grad()

        # Forward pass
        encoded, decoded = autoencoder(inputs)

        # Compute the MSE loss by comparing the decoded output with the input
        loss = criterion(decoded, inputs)

        # Add the regularization term
        loss += 0.000001 * (decoded ** 2).sum()

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


Epoch [1/20], Loss: 0.07313347607851028
Epoch [2/20], Loss: 0.06859830766916275
Epoch [3/20], Loss: 0.07338949292898178
Epoch [4/20], Loss: 0.062432266771793365
Epoch [5/20], Loss: 0.07072839885950089
Epoch [6/20], Loss: 0.10643003135919571
Epoch [7/20], Loss: 0.10137803107500076
Epoch [8/20], Loss: 0.11038287729024887
Epoch [9/20], Loss: 0.11204873025417328
Epoch [10/20], Loss: 0.10847843438386917
Epoch [11/20], Loss: 0.11162326484918594
Epoch [12/20], Loss: 0.11785528808832169
Epoch [13/20], Loss: 0.11236056685447693
Epoch [14/20], Loss: 0.11572087556123734
Epoch [15/20], Loss: 0.11039432138204575
Epoch [16/20], Loss: 0.11167068779468536
Epoch [17/20], Loss: 0.11208735406398773
Epoch [18/20], Loss: 0.10728941857814789
Epoch [19/20], Loss: 0.10215115547180176
Epoch [20/20], Loss: 0.11073541641235352
